In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tenseal as ts
import torch
from sklearn.model_selection import train_test_split
from time import time

#### Обучение на незашифрованных данных

In [2]:
df = pd.read_csv('prepared_data.csv', low_memory=False)

In [6]:
x = df.drop(columns=['Is_Delinquent'])
y = df['Is_Delinquent'].copy()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [7]:
x_train = torch.tensor(x_train.values).to(torch.float32)
x_test = torch.tensor(x_test.values).to(torch.float32)
y_train = torch.tensor(y_train.values).to(torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test.values).to(torch.float32).unsqueeze(1)

In [8]:
class LogisticRegression(torch.nn.Module):

    def __init__(self, n_features):
        super(LogisticRegression, self).__init__()
        self.lr = torch.nn.Linear(n_features, 1)
        
    def forward(self, x):
        out = torch.sigmoid(self.lr(x))
        return out
    
n_features = x_train.shape[1]
model = LogisticRegression(n_features)
optim = torch.optim.SGD(model.parameters(), lr=1)
criterion = torch.nn.BCELoss()

In [9]:
EPOCHS = 3

def train(model, optim, criterion, x, y, epochs=EPOCHS):
    for e in range(1, epochs + 1):
        optim.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optim.step()
        print(f'Среднее значение функции потерь за эпоху {e}: {loss.data}')
    return model

model = train(model, optim, criterion, x_train, y_train)

Среднее значение функции потерь за эпоху 1: 0.7707149982452393
Среднее значение функции потерь за эпоху 2: 0.33744561672210693
Среднее значение функции потерь за эпоху 3: 0.3344480097293854


In [10]:
def accuracy(model, x, y):
    out = model(x)
    correct = torch.abs(y - out) < 0.5
    return correct.float().mean()

plain_accuracy = accuracy(model, x_test, y_test)
print(f'Точность модели обученной на незашифрованных данных: {plain_accuracy}')

Точность модели обученной на незашифрованных данных: 0.8409090638160706
